In [28]:
import numpy as np
import glob
import os
import re
# nltk.download() if not installed nltk
from nltk.corpus import stopwords  # Stopwords: ‘the’, ‘is’, ‘are’...
from nltk.stem.porter import * # Stem: gamer, gaming, game -> game
from nltk.tokenize import RegexpTokenizer # Regexp: set rule to just tokenize word

### Load Feature

In [29]:
train_feature, test_feature = [], []

In [30]:
for line in open("./data/features_train/features_resnet1000_train.csv"):
    tokens = line.strip().split(",")
    train_feature.append(tokens)
train_feature = np.array(train_feature)

In [31]:
for line in open("./data/features_test/features_resnet1000_test.csv"):
    tokens = line.strip().split(",")
    test_feature.append(tokens)
test_feature = np.array(test_feature)

### Load Description

#### Global Dictionary

In [32]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

In [33]:
def create_dict(path):
    dictionary = {}
    
    ## glob.glob return paths matching given pattern
    for file in glob.glob(os.path.join(path, "*.txt")):
        with open(file, "r") as file_content:
            content = file_content.read()
            content = np.char.lower(content) # Lowercase
            content = re.sub('[^\w\s]', ' ', str(content)) # Define split
            
            for word in tokenizer.tokenize(content): # Remove punctuation
                try:
                    if word not in stop_words: # Remove stopwords
                        w = stemmer.stem(word) # Stem
                        if w in dictionary:
                            dictionary[w] += 1
                        else:
                            dictionary[w] = 1
                except:
                    pass 
    return dictionary

In [34]:
global_desc_dict = create_dict("./data/descriptions_train/")

In [35]:
global_desc_dict

{'teddi': 402,
 'bear': 960,
 'cloth': 141,
 'hang': 367,
 'line': 381,
 'outsid': 708,
 'window': 687,
 'stuf': 341,
 'toy': 204,
 'laundri': 5,
 'item': 245,
 'left': 75,
 'air': 615,
 'dri': 116,
 'pin': 21,
 'outdoor': 216,
 'cat': 1591,
 'ground': 388,
 'shoe': 128,
 'kitten': 77,
 'play': 1437,
 'lace': 3,
 'pair': 266,
 'blue': 1174,
 'shoelac': 2,
 'floor': 492,
 'grey': 143,
 'tabbi': 25,
 'navi': 12,
 'string': 36,
 'gray': 168,
 'tiger': 17,
 'walk': 1547,
 'across': 300,
 'brick': 299,
 'two': 4028,
 'women': 380,
 'togeth': 462,
 'near': 1684,
 'tree': 1190,
 'pretti': 76,
 'exot': 7,
 'dress': 327,
 'robe': 19,
 'orient': 17,
 'costum': 46,
 'onlook': 25,
 'nearbi': 80,
 'walkway': 38,
 'geisha': 1,
 'tourist': 23,
 'area': 810,
 'coupl': 837,
 'rocki': 50,
 'surfac': 104,
 'wooden': 697,
 'cut': 396,
 'board': 755,
 'knife': 169,
 'carrot': 215,
 'onion': 105,
 'assort': 108,
 'veget': 434,
 'made': 152,
 'interest': 13,
 'shape': 126,
 'brown': 727,
 'tabl': 2606,
 'blo

#### Bag Of Word

In [36]:
len(global_desc_dict)

6456

In [37]:
def create_desc_bow(file_path, global_dict, threshold):
    dict_thresh = {}   
    i = 0
    for w in global_dict:
        if global_dict[w] >= threshold:
            dict_thresh[w] = i
            i = i + 1
    
    dict_freq = []
    for i in range(10000):
        file_name = str(i) + ".txt"
        file = os.path.join(file_path, file_name)
        with open(file, "r") as file_content:
            word_dict_freq = {}
            
            content = file_content.read()
            content = np.char.lower(content) # Lowercase
            content = re.sub('[^\w\s]', ' ', str(content)) # Define split
            
            for word in tokenizer.tokenize(content): # Remove punctuation
                try:
                    if word not in stop_words: # Remove stopwords
                        w = stemmer.stem(word) # Stem
                        if w in dict_thresh:
                            if w in word_dict_freq:
                                word_dict_freq[w] += 1
                            else:
                                word_dict_freq[w] = 1
                except:
                    pass
            dict_freq.append(word_dict_freq)
            
    return dict_freq             

In [38]:
desc_dict_freq = create_desc_bow("./data/descriptions_train/", global_desc_dict, 20)

In [39]:
type(desc_dict_freq)

list

In [40]:
desc_dict_freq[1]

{'asian': 1,
 'bowl': 4,
 'carrot': 3,
 'eat': 1,
 'enjoy': 1,
 'food': 1,
 'next': 1,
 'noodl': 2,
 'readi': 1,
 'serv': 1,
 'sit': 1,
 'someon': 1,
 'soup': 3,
 'tasti': 1}

### Load Tag

In [41]:
def create_dict_tags(path):
    tag_dict = {}
    for i in range(10000):
        filename = str(i) + ".txt"
        file = os.path.join(path, filename)
        with open(file, "r") as file_content:
            content = file_content.read()
            
            for line in content.splitlines():
                for idx,token in enumerate(line.split(':')):
                    if idx > 0 and token not in stop_words:
                        if token in tag_dict:
                            tag_dict[token] += 1
                        else:
                            tag_dict[token] = 1
    return tag_dict

In [42]:
tag_dict = create_dict_tags("./data/tags_train")

In [43]:
len(list_tags)

NameError: name 'list_tags' is not defined

In [44]:
def getTop(dict,n):
    s = (sorted(tag_dict.items(), key=lambda x:x[1]))[-n:]
    
    l = []
    for e in s:
        l.append(e[0])
    return l

In [45]:
top_20_tag = getTop(tag_dict, 5)

In [46]:
top_20_tag

['tv',
 'cat',
 'potted plant',
 'umbrella',
 'sink',
 'dog',
 'backpack',
 'cell phone',
 'clock',
 'book',
 'bench',
 'truck',
 'bowl',
 'handbag',
 'bottle',
 'cup',
 'dining table',
 'car',
 'chair',
 'person']

In [50]:
top_5_keyword_for_0 = getTop(desc_dict_freq[0], 5)

In [51]:
top_5_keyword_for_0

['cup', 'dining table', 'car', 'chair', 'person']